In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import datetime
import os

In [3]:
ads_data=pd.read_csv('olx_data/ads_data/ads_data.csv')
user_data=pd.read_csv('olx_data/user_data/user_data.csv')
user_msg=pd.read_csv('olx_data/user_messages.csv')
test=pd.read_csv('olx_data/user_messages_test.csv')

In [4]:
len(pd.unique(ads_data['category_id']))

10

In [5]:
test.shape
user_msg.shape
ads_data.shape
user_data.shape


(10507, 2)

(10348, 3)

(645168, 11)

(2143889, 12)

In [6]:
#no common users in test and user_msg.csv
lst1=set(test['user_id'])
len(lst1)
lst2=set(user_msg['user_id'])
len(lst2)

len(lst1.intersection(lst2))

7162

7161

0

In [7]:
test.head(2)
user_msg.head(2)
ads_data.head(2)
user_data.head(2)

,user_id,category_id
0,2,800
1,6,815


,user_id,category_id,ads
0,1,859,[1806476]
1,3,800,"[2131700, 2734107, 2877209, 2877209]"


,ad_id,category_id,seller_id,creation_time,title,description,price,lat,long,source,enabled
0,1344,800,1157,2015-11-29 13:05:26,Blackberry curve,"Funcionando, es para personal el único problem...",300.0,NaN,NaN,android,0
1,3916,815,59,2015-12-16 09:09:25,Remera marca premium alemana,Casi nuevo y sin uso. Lo compre online y me qu...,150.0,NaN,NaN,android,1


,event_time,user_id,event,channel,user_lat,user_long,origin,ad_id,images_count,ad_impressions,ad_views,ad_messages
0,2017-05-19 10:47:22,541,view,ios,-34.600439,-58.514031,browse_search,2426321,6.0,0.0,4.0,0.0
1,2017-06-10 10:18:12,501,view,android,-34.818047,-58.356583,browse_search,2746814,5.0,152.0,85.0,1.0


In [8]:
user_data['event_time']=pd.to_datetime(user_data['event_time'])
# user_data['event_time']=pd.to_datetime(str(user_data['event_time'].dt.month)+'-'+str()
user_data['month']=user_data['event_time'].dt.month
user_data['day']=user_data['event_time'].dt.day

In [9]:
cols=['channel','user_lat','user_long']
user_data.drop(cols,axis=1,inplace=True)

mapping = {'view':0 , 'first_message':1}
user_data = user_data.replace({'event':mapping})

mapping = {'browse_search':0,'browse':0 ,'search':0,'notification_center':1, 'home':2, 'push':3, 'drawer':4, 'deeplink':4 }
user_data = user_data.replace({'origin':mapping})

user_data.dtypes

event_time        datetime64[ns]
user_id                    int64
event                      int64
origin                   float64
ad_id                      int64
images_count             float64
ad_impressions           float64
ad_views                 float64
ad_messages              float64
month                      int64
day                        int64
dtype: object

In [10]:
# create ads and category dataframe with ads_id as index
ind=np.array(ads_data['ad_id'])
cat=np.array(ads_data['category_id'])
ads_cat=pd.DataFrame(cat,index=ind,columns=['category_id'])

price=np.array(ads_data['price'])
ads_price=pd.DataFrame(price,index=ind,columns=['price'])

enabled=np.array(ads_data['enabled'])
ads_enabled=pd.DataFrame(enabled,index=ind,columns=['enabled'])



user=np.array(user_data['user_id'])
ind=np.array(user_data['ad_id'])

ads_user=pd.DataFrame(user,index=ind,columns=['user'])
user_ads=pd.DataFrame(ind,index=user,columns=['ad_id'])

ad_info = pd.DataFrame(np.array(user_data),index=ind,columns=user_data.columns)

In [11]:
pd.unique(user_data['origin'])

array([  0.,   2.,   1.,   3.,  nan,   4.])

In [12]:
user_data = user_data.sort_values(by=['month','day','origin','ad_messages','ad_views','ad_impressions'],ascending=[False,False,True,False,False,False])

In [13]:
ans={}
cat=pd.unique(test['category_id'])
for cid in cat:
    df1=user_data
    
    
    catdf = ads_cat.loc[ df1['ad_id'] ]
    df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
    
    
    enabledf = ads_enabled.loc[ df1['ad_id'] ]
    df1=df1[np.array(enabledf==1)]

    
    lst=[]
    #to get unique elements in lst
    ids=list(df1['ad_id'])
    for i in np.arange(len(ids)):
        if len(lst)==10:
            break
        if not ids[i] in lst:
            lst.append(ids[i])

    ans[cid]=lst
#     print df1.columns

In [ ]:
# np.unique(ads_user.loc[2426321])
# user_ads.loc[541]

In [ ]:
j=0
result=test[['user_id','category_id']]
res_ads=[]
for i in test.index:
    if j%10==0:
        print j
    j = j+1
    row=test.loc[i]
    uid=row['user_id']
    cid=row['category_id']

    df1 = user_data.loc[ user_data['user_id']==uid ]
    
    related_user = np.unique(ads_user.loc[ df1['ad_id'] ])
    related_ads = np.unique(user_ads.loc[related_user])
    
    df1 = ad_info.loc[related_ads]
    
    if df1.shape[0]==0:
        res_ads.append(str(ans[cid]))
        continue
        
    catdf = ads_cat.loc[ df1['ad_id'] ]
    df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
    
    enabledf = ads_enabled.loc[ df1['ad_id'] ]
    if df1.shape[0]>0:
        df1=df1[np.array(enabledf==1)]
#     df1=df1.sort_values(by=['ad_messages','ad_views','ad_impressions'],ascending=[False,False,False])
    
    lst=[]
    lst=list(df1['ad_id'][0:10])
    
    if len(lst)<10:
        rem=10-len(lst)
#         rem
#         ans[cid]
        lst.extend(ans[cid][0:rem])
        
    res_ads.append(str(lst))
    
#     print 'harshkara'
#     break
    

0
10
20
30
40
50
60
70
80
90
100


In [ ]:
type(related_ads)

In [ ]:
result['ads']=res_ads

In [ ]:
result.to_csv("res_meta_path.csv", index=False)
os.system("spd-say 'task complete'")

In [ ]:
#start time 9:20pm

In [ ]:
# lst=[1,2,3,4,5]
# lst.extend([7,8,9])
# lst

In [ ]:
# list(df1['ad_id'][0:10])

In [ ]:
# df['e']=df['e']+np.ones(df.shape[0],dtype=int)
# df

In [ ]:
# a=np.array([[1,2,3,4,5],[6,7,8,9,10],[1,4,2,6,7],[3,1,6,9,2],[4,5,6,7,8]])
# df=pd.DataFrame(a,columns=['a','b','c','d','e'])
# df[0:100]
# b=[1,2,3]
# # ind=[0,2,4]
# df=df.loc[df['a']==1]
# df
# df['xyz']=np.array([99,100])
# df

In [ ]:
# tdf=pd.DataFrame(np.array(b),index=ind,columns=['xyz'])
# tdf
# tdf==3
# oop=[7,45]
# tdf.loc[oop]

In [ ]:
# df
# tdf
# tdf.loc[tdf['xyz']==3]


In [ ]:
# df['xyz']=np.array(tdf)
# df

In [ ]:
# df.loc[np.logical_and(df['a']==1, np.mod(df['d'],2)==1)]

In [ ]:
# #ONLY FOR TESTING PURPOSE
# # uid=row['user_id']
# uid=18
# cid=800
# # cid=row['category_id']
# print uid
# print cid
# #     user_cat_ads = user_data.loc[ np.logical_and( user_data['user_id']=uid , user_data['']  )    ]
# df1 = user_data.loc[ user_data['user_id']==uid ]
# df1
# if df1.shape[0]==0:
#     df1=user_data

# catdf = ads_cat.loc[ df1['ad_id'] ]
# print '*******************************'
# catdf
# df1 = df1[ np.array(catdf==cid) ]  #selecting those adds that have the same category as cid i.e. test instance
# pricedf = ads_price.loc[ df1['ad_id'] ]

# df1['price'] = np.array(pricedf)
# df1